## Workbook to read in data from different sources and prep for processing

In [1]:
import geopandas as gp
import pandas as pd
import numpy as np
import os
import glob
import requests
import warnings
import uuid

## Flares data

In [2]:
projcrs = 4326

In [3]:
def read_excel(sheet_name):
    
    all_files = glob.glob("data/rmidata/*.xlsx")
    temp = []

    for filename in all_files:
        df = pd.read_excel(filename, index_col=None, header=0, sheet_name=sheet_name)
        temp.append(df)

    df = pd.concat(temp, axis=0, ignore_index=True)

    # some data cleanup
    df.rename(columns={'BCM 2019':'BCM_2019', 'Avg. temp':'avg_temp',
                       'BCM 2018':'BCM_2018', 'BCM 2020':'BCM_2020',
                       'BCM 2021':'BCM_2021',
                       'Detection_frequency_2012':'Det_freq12',
                       'Detection_frequency_2013':'Det_freq13',
                       'Detection_frequency_2014':'Det_freq14',
                       'Detection_frequency_2015':'Det_freq15',
                       'Detection_frequency_2016':'Det_freq16',
                       'Detection_frequency_2017':'Det_freq17',
                       'Detection frequency 2018':'Det_freq18',
                       'Detection freq. 2019':'Det_freq19',
                       'Detection frequency 2020':'Det_freq20',
                       'Detection frequency 2021':'Det_freq21',
                       'Clear_obs_2012': 'clr_obs12',
                       'Clear_obs_2013': 'clr_obs13',
                       'Clear_obs_2014': 'clr_obs14',
                       'Clear_obs_2015': 'clr_obs15',
                       'Clear_obs_2016': 'clr_obs16',
                       'Clear_obs_2017': 'clr_obs17',
                       'Clear obs 2018': 'clr_obs18',
                       'Clear obs. 2019': 'clr_obs19',
                       'Clear Obs. 2020': 'clr_obs20',
                       'Clear Obs. 2021': 'clr_obs21'}, inplace=True)
    
    df["flare_category"] = sheet_name  # add new column for overall flare type, to make summary stats easier
    
    return(df)

In [4]:
def convert_geopandas(df):
    '''Convert pandas df to geodataframe, drop orig Lat/Long cols, and set crs to common project crs'''
    
    df = gp.GeoDataFrame(
        df, geometry=gp.points_from_xy(df["Longitude"],
                                       df["Latitude"]), 
        crs = projcrs)

    #df.drop(['Latitude', 'Longitude'], axis=1, inplace=True) 
    return(df)

In [5]:
# define sheet names
all_upstream="flares_upstream"
oil_downstream="flares_oil_downstream"
gas_downstream="flares_gas_downstream"

# create dfs by flare category
flares_upstream = read_excel(all_upstream)
oil_downstream = read_excel(oil_downstream)
gas_downstream = read_excel(gas_downstream)

# combine into single df and convert to geodataframe
all_flares = convert_geopandas(pd.concat([flares_upstream, oil_downstream, gas_downstream]))

In [6]:
# not all flares have a unique identifier — both CatalogID and ID columns have missing values
# so I create a new col "flare_id" of IDs

np.random.seed(123)

# generate random integer IDs
ids = np.random.randint(100000, 999999, size=len(all_flares))

# convert integer IDs to unique string identifiers
id_str = [str(uuid.uuid4())[:8] + str(i) for i in ids]

# add the new column to the DataFrame
all_flares['flare_id'] = id_str

In [7]:
all_flares.sample(5)

,Country,ISO Code,Catalog ID,id #,Latitude,Longitude,BCM_2019,avg_temp,Ellipticity,Detection_frequency_2019,...,Det_freq16,Catalog id,BCM_2018,Det_freq18,clr_obs18,flare_category,Detection freq.,Det_freq19,geometry,flare_id
17755,Libya,LBY,LBY_UPS_2018_19.9139E_28.2088N_v0.2,NaN,28.208772,19.913904,NaN,NaN,1.35255,NaN,...,NaN,NaN,NaN,NaN,NaN,flares_upstream,NaN,NaN,POINT (19.91390 28.20877),c78ef7d5428785
32513,Russian Federation,NaN,NaN,NaN,62.077548,71.025300,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,flares_upstream,NaN,NaN,POINT (71.02530 62.07755),a21d2656778989
751,Russian Federation,RUS,RUS_UPG_2015_76.8498E_65.9371N_v0.2,7348.0,65.937147,76.849779,0.001154,1712.17,1.60150,NaN,...,NaN,NaN,NaN,NaN,NaN,flares_oil_downstream,1.86916,NaN,POINT (76.84978 65.93715),f8792d5e368489
17197,United States,USA,USA_UPS_2019_96.2354W_30.7334N_v0.2,NaN,30.733352,-96.235449,NaN,NaN,1.60150,NaN,...,NaN,NaN,NaN,NaN,NaN,flares_upstream,NaN,NaN,POINT (-96.23545 30.73335),e1699316185509
25560,China,CHN,CHN_UPS_2016_108.2411E_36.0699N_v0.2,NaN,36.069889,108.241148,NaN,NaN,1.33265,NaN,...,NaN,NaN,NaN,NaN,NaN,flares_upstream,NaN,NaN,POINT (108.24115 36.06989),a3eee7d4186002


In [8]:
# save as shapefile
all_flares.to_file("data/all_flares.shp")

/Users/natebender/.local/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  


## Social vulnerability data from CalEnviroScreen

No cleaning necessary; shapefile read in directly to analysis notebooks

## Refineries

No cleaning necessary; csv read in directly to analysis notebooks

## EJScreen data: ACS 5-year Estimates (2017-2021)

In [9]:
# Load census block group level data from EJscreen
# Source: https://www.epa.gov/ejscreen/download-ejscreen-data
# This is the 2017-2021 5-year ACS average data
ejscreen = pd.read_excel("data/CA_EJSCREEN_2022_Full_with_AS_CNMI_GU_VI.xlsx", index_col=None, header=0)

ca_counties = gp.read_file("data/CA_Counties_TIGER2016.shp")  # CA counties
ca_bg = gp.read_file("data/tl_2022_06_bg.shp")  # CA block groups

# read in cleaned and combined flares data
all_flares = gp.read_file("data/all_flares.shp")

In [10]:
meters_crs = 3310  # Projected crs. this should be good for this overlay() calculation and all of project. 

ca_counties = ca_counties.to_crs(meters_crs)
ca_bg = ca_bg.to_crs(meters_crs)
all_flares = all_flares.to_crs(meters_crs)

In [11]:
# Merge EJscreen data with CA block groups shapefile
ca_bg.rename(columns={'GEOID':'ID'}, inplace=True)  # old:new. match column names for merging
ca_bg['ID'] = ca_bg['ID'].astype(np.int64)

print(f"{len(ejscreen['ID'])} block groups in the EJScreen data \n")
print(f"{len(ca_bg['ID'])} block groups in the CA block group shapefile\n")
ca_bg_joined = pd.merge(ca_bg, ejscreen, on='ID')
print(f"{len(ca_bg['ID'].unique())-len(ca_bg_joined)} block groups are missing after merge")
warnings.filterwarnings('ignore')

# create column of correct area
ca_bg_joined.loc[:, 'shape_area_new'] = ca_bg_joined.geometry.area
ca_bg_joined.rename(columns={'ID':'bg_id'}, inplace=True)  # old:new
ca_bg_joined.rename(columns={'CNTY_NAME':'cnty_name'}, inplace=True)  # old:new.

25607 block groups in the EJScreen data 

25607 block groups in the CA block group shapefile

0 block groups are missing after merge


In [12]:
# A buffer around the California coastline is included in the block groups but shouldn't be in the analysis. 
# Removing these block groups.

ej_nan = ca_bg_joined[ca_bg_joined['D_PM25_2'].isnull()]
print(len(ej_nan))
ej_nan.explore()

17


In [13]:
ids_to_drop = list(ej_nan['bg_id'])
# ids_to_drop = [60839900000, 61119901000, 60379902000, 60379903000, 60599901000]

# Drop the rows with those IDs
ca_bg_joined = ca_bg_joined[~ca_bg_joined['bg_id'].isin(ids_to_drop)]

In [14]:
len(ca_bg_joined)

25590

## Subset flares to CA only

In [16]:
# subset flares to only those in Cali
ca_flares = gp.sjoin(all_flares, ca_counties, how = "inner", predicate = 'within')
print(f'Flares found: {len(ca_flares)}')

ca_flares.rename(columns={'NAMELSAD':'cnty_name'}, inplace=True)  # old:new. Match col names for merging
ca_flares.drop('index_right', axis=1, inplace=True)

Flares found: 117


In [17]:
print(all_flares['flare_cate'].unique())

# rename the flare categories for readability
ca_flares["flare_cate"] = (ca_flares["flare_cate"].
                                    replace({"flares_oil_downstream": "Refinery",
                                             "flares_upstream": "Upstream"}))

# reminder: once you subset to CA there are no downstream gas flares
print(ca_flares['flare_cate'].unique())

['flares_upstream' 'flares_oil_downstream' 'flares_gas_downstream']
['Upstream' 'Refinery']


In [18]:
# set col list for BCM_avg calculation
bcm_list = ['BCM_2012','BCM_2013','BCM_2014','BCM_2015','BCM_2016','BCM_2017',
            'BCM_2018','BCM_2019','BCM_2020','BCM_2021']

# add new column for average BCM across all years
ca_flares['BCM_avg'] = ca_flares[bcm_list].mean(axis=1) 

## Save files

In [19]:
ca_flares.to_file('data/ca_flares_clean.shp', driver='ESRI Shapefile')

In [20]:
ca_bg_joined.to_file('data/ca_bg_joined_clean.shp', driver='ESRI Shapefile')

In [21]:
ca_bg.to_file('data/ca_bg_clean.shp', driver='ESRI Shapefile')

In [22]:
# For Tableau mapping break into shapefile and text file, aligned by flare_id: 
ca_flarestableau_geo = gp.GeoDataFrame(ca_flares[['flare_id', 'geometry']].copy(), 
                                      geometry='geometry', crs=meters_crs)


ca_flarestableau_geo.to_file('data/df_flare_caflares_geo.shp', driver='ESRI Shapefile')

## API: 2010 US Census data

### 2/16/23 note: not using census API right now. Maybe later in project. 

Czolowsky et al used: 
total population, Hispanic, minority, non-Hispanic minority, 5y and younger, under 18y, and 75y and older.

Var list

P001001: total population

P004001: total hispanic or latino origin

H008002: Total races tallied for householders — White alone or in combination with one or more other races


In [ ]:
# keys = pd.read_csv("api_keys.csv")
# census_api_key = keys.loc[keys['api']=='census']['key'].iloc[0]

In [ ]:
# # helpful vid: https://www.youtube.com/watch?v=LW-M_UC0VTE
# # dec/sf1 variables: https://api.census.gov/data/2010/dec/sf1/variables.html
# # dec/sf1 example api calls: https://api.census.gov/data/2010/dec/sf1/examples.html
# # CA FIPS codes: https://www.weather.gov/hnx/cafips
# # FYI California has 8,057 census tracts, 23,212 block groups, and 710,145 blocks

# # set search query and parameters
# url = "https://api.census.gov/data/2010/dec/sf1?get=NAME,TRACT,P001001,P004001,H008002&for=block:*&in=state:06&in=county:001&in=tract:*&key={0}".format(census_api_key)

# # make call to api
# response = requests.request("GET", url)

In [ ]:
# # clean up column names that come from the api
# col_names = ["name", "tract_id", "total_pop", "total_hisp", "race_white", "state", "county", "tract", "block"]

# # construct df, beg. at first row so headers are skipped
# df = pd.DataFrame(columns=col_names, data=response.json()[1:]) 

In [ ]:
# # check a few rows
# df.sample(5)

In [ ]:
# # save to a csv for further processing in other notebooks
# df.to_csv("data/ca_blocklevel_demographics.csv")